In [1]:
from pamona import Pamona
import numpy as np
import pamona

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
import sys
sys.path.insert(1, '2019_mmd_wabi')
import manifoldAlignDistortionPen_mmd_multipleStarts as mmdma

In [1]:
import os
os.environ['R_HOME'] = '/risapps/rhel7/R/3.6.0/lib64/R' #path to your R installation
os.environ['R_LIBS_USER'] = "/rsrch3/home/bcb/sliang3/R/x86_64-pc-linux-gnu-library/3.6:/risapps/rhel7/R/3.6.0/lib64/R/library"
import rpy2
%load_ext rpy2.ipython

In [ ]:
suffix = '0.7'
file_name = 'Ncell2000_PopSize_nGene500_nPeak1500_w' + suffix + '.RDS'

In [2]:
%%R -o rna -o atac -i file_name
data <- readRDS(file_name)
rna <- data$X
atac <- data$Y

In [3]:
print(rna.dim)
print(atac.dim)

[1]  500 2000

[1] 1500 2000



In [7]:
#random_index = np.random.choice(30672, 5000, False)

data1 = np.array(rna).T #[random_index, :]
data2 = np.array(atac).T #[random_index, :]

shuffle = np.arange(data2.shape[0])
np.random.shuffle(shuffle)
shuffle_back = np.argsort(shuffle)

data2 = data2[shuffle, :]

In [8]:
from sklearn.decomposition import PCA

##type1 = LabelEncoder().fit_transform(meta['celltype1'])#[random_index]
##type2 = LabelEncoder().fit_transform(meta['celltype1'])#[random_index]

data1 = pamona.utils.zscore_standardize(np.asarray(data1))
data1 = PCA(30).fit_transform(data1)
#data1 = pamona.utils.zscore_standardize(data1)

data2 = pamona.utils.zscore_standardize(np.asarray(data2))
data2 = PCA(30).fit_transform(data2)
#data2 = pamona.utils.zscore_standardize(np.asarray(data2))

data = [data1,data2]
#datatype = [type1,type2]

In [9]:
%%R

rm(data)
gc()

          used (Mb) gc trigger (Mb) max used (Mb)
Ncells  297252 15.9     641825 34.3   514015 27.5
Vcells 2903873 22.2    8388608 64.0  4919946 37.6


In [10]:
data1.shape

(3000, 30)

In [11]:
data2.shape

(800, 30)

In [ ]:
from scipy.spatial.distance import pdist, squareform
k1 = squareform(pdist(data1))
k2 = squareform(pdist(data2))

integrated_data = mmdma.mmd_ma(k1, k2, max_iter=25001)

In [ ]:
embedding = pd.DataFrame(integrated_data[0],
                         columns=['X_' + str(i) for i in range(integrated_data[0].shape[1])])
embedding.to_csv("output/sim4-" + suffix + "-mmdma-rna.csv")

embedding = pd.DataFrame(integrated_data[1][shuffle_back, :],
                         columns=['Y_' + str(i) for i in range(integrated_data[1].shape[1])])
embedding.to_csv("output/sim4-" + suffix + "-mmdma-atac.csv")